In [1]:
import pandas as pd
import numpy as np
import calendar
import seaborn as sns
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTENC
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
# pip install git+https://github.com/scikit-learn-contrib/category_encoders.git#egg=httpie
# pip install imblearn

Using TensorFlow backend.


## **刪除沒有被保人資料的列**

In [4]:
df = pd.read_excel('理賠再購屬性合併1_2.xlsx',index_col=[0,1])

In [12]:
df = df.dropna(subset=['CUST_RK'])

In [13]:
df.to_excel('理賠再購屬性合併1-2_before_encoding.xlsx')

## **匯入資料**

In [2]:
df = pd.read_excel('理賠再購屬性合併1-2_before_encoding.xlsx',index_col=[0,1])

## **特徵工程**

In [3]:
df.columns

Index(['INJURED_RK', 'Claim_RK', 'BundleSubtype2', 'illness_code',
       'illness_desc', 'DiagnosisCode_DESC', 'claim_settle_dt',
       'REIMBURSED_YR_TW', 'POLICY_HOLDER_RK', 'MATURITY_BENEFICIARY_RK',
       'DEATH_BENEFICIARY_RK', '結案後120天', '結案後180天', '結案後360天', '再購(120天)',
       '再購(180天)', '再購(360天)', 'CUST_RK', 'ternure_m', 'recency_m', 'SIN',
       'SIN_his', 'REG', 'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb',
       'AHb_his', 'AHc', 'AHc_his', 'AHd', 'AHd_his', 'VIP_CLASS', 'VIP',
       'WEALTH_LEVEL', 'CLIENT_MARITAL', 'CLIENT_INCOME', 'DIGI_FLG',
       'TOPCARD', 'GENDER', 'stick_level2', 'cust_group2', 'TOTAL_AUM',
       'INSURED_DOB'],
      dtype='object')

In [4]:
df['被保人是否為事故人'] = np.where(df['INJURED_RK']==df.index.get_level_values(0), 1, 0)
df['結案月份'] = list(map(lambda x:str(x)[5:7],df['claim_settle_dt']))
df['計數'] = 1
df['累積理賠金額'] = df.groupby('Policy_RK')['REIMBURSED_YR_TW'].transform('cumsum')
df['初次理賠時間'] = df.groupby('Policy_RK')['claim_settle_dt'].transform(min)
df['累積理賠次數'] = df.groupby('Policy_RK')['計數'].transform('cumsum')
df = df.drop(columns=['計數'])
df['被保人年收'] = np.where(df['CLIENT_INCOME']>=df['CLIENT_INCOME'].median(), 1, 0) #好像有直接分幾等份的method
df['被保人總資產'] = np.where(df['TOTAL_AUM']>=df['TOTAL_AUM'].median(), 1, 0)
df['具滿期金受益人'] = np.where(df['MATURITY_BENEFICIARY_RK'].isna(), 0, 1)
df['具生故保險金受益人'] = np.where(df['DEATH_BENEFICIARY_RK'].isna(), 0, 1)

In [5]:
df = df[['再購(120天)',
       '再購(180天)', '再購(360天)','INJURED_RK', 'Claim_RK', 'BundleSubtype2', 'illness_code',
       'illness_desc', 'DiagnosisCode_DESC', 'claim_settle_dt',
       'REIMBURSED_YR_TW', 'POLICY_HOLDER_RK', 'MATURITY_BENEFICIARY_RK',
       'DEATH_BENEFICIARY_RK', '結案後120天', '結案後180天', '結案後360天', 'CUST_RK', 'ternure_m', 'recency_m', 'SIN',
       'SIN_his', 'REG', 'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb',
       'AHb_his', 'AHc', 'AHc_his', 'AHd', 'AHd_his', 'VIP_CLASS', 'VIP',
       'WEALTH_LEVEL', 'CLIENT_MARITAL', 'CLIENT_INCOME', 'DIGI_FLG',
       'TOPCARD', 'GENDER', 'stick_level2', 'cust_group2', 'TOTAL_AUM',
       'INSURED_DOB', '被保人是否為事故人', '結案月份', '累積理賠金額', '初次理賠時間', '累積理賠次數',
       '被保人年收', '被保人總資產', '具滿期金受益人', '具生故保險金受益人']]

In [6]:
df

再購(120天)  再購(180天)  再購(360天)  INJURED_RK  Claim_RK  \
INSURED_RK Policy_RK                                                       
152        131337            0         0         0         129    104154   
158        124517            0         0         0         158     51549   
           124517            0         0         0         158     62175   
           124517            0         0         0         158    149340   
204        153186            0         0         0         156    138472   
...                        ...       ...       ...         ...       ...   
251796     74433             0         0         0       82163    189132   
251813     78696             0         0         0       26496    117881   
251876     57631             0         0         0      105960     80566   
251904     105696            1         1         1      225469    189078   
251954     74765             0         0         0      156320    136040   

                     BundleSubtype2 illness_code          illness_desc  \
INSURED_RK Policy_RK                                                     
152        131337           4.C重大疾病          I63                   腦梗塞   
158        124517           5.N疾病醫療        J04.1                 急性氣管炎   
           124517           5.N疾病醫療        J98.9                呼吸系統疾患   
           124517           5.N疾病醫療          M51       胸椎、胸腰椎及腰薦椎椎間盤疾患   
204        153186           5.N疾病醫療        N20.9             泌尿道結石，未特定   
...                             ...          ...                   ...   
251796     74433            6.N意外醫療          V23  摩拖車騎士與汽車，小貨車或箱型車碰撞受傷   
251813     78696            6.N意外醫療          V23  摩拖車騎士與汽車，小貨車或箱型車碰撞受傷   
251876     57631            6.N意外醫療        Y93.7        涉及其他特定運動和競技之活動   
251904     105696           5.N疾病醫療          D09          其他及未明示部位之原位癌   
251954     74765            4.C重大疾病        I25.1        自體的冠狀動脈粥樣硬化心臟病   

                     DiagnosisCode_DESC claim_settle_dt  ...  INSURED_DOB  \
INSURED_RK Policy_RK                                     ...                
152        131337             09.循環系統疾病      2017-01-20  ...   1951-02-26   
158        124517             10.呼吸系統疾病      2016-01-16  ...   1978-12-09   
           124517             10.呼吸系統疾病      2016-04-24  ...   1978-12-09   
           124517      13.肌肉骨骼系統和結締組織疾病      2017-06-04  ...   1978-12-09   
204        153186           14.泌尿生殖系統疾病      2017-05-05  ...   1976-09-20   
...                                 ...             ...  ...          ...   
251796     74433            20.疾病和死亡的外因      2017-09-16  ...   1966-11-27   
251813     78696            20.疾病和死亡的外因      2017-03-10  ...   1966-03-07   
251876     57631            20.疾病和死亡的外因      2016-09-18  ...   1975-02-26   
251904     105696                 02.腫瘤      2017-09-16  ...   1964-04-21   
251954     74765              09.循環系統疾病      2017-04-29  ...   1960-09-28   

                      被保人是否為事故人  結案月份    累積理賠金額     初次理賠時間 累積理賠次數 被保人年收  \
INSURED_RK Policy_RK                                                      
152        131337             0    01  23100.00 2017-01-20      1     0   
158        124517             1    01  30326.10 2016-01-16      1     0   
           124517             1    04  67120.20 2016-01-16      2     0   
           124517             1    06  86020.20 2016-01-16      3     0   
204        153186             0    05  21462.00 2017-05-05      1     0   
...                         ...   ...       ...        ...    ...   ...   
251796     74433              0    09   4389.00 2017-09-16      1     0   
251813     78696              0    03  31133.55 2017-03-10      1     0   
251876     57631              0    09   1470.00 2016-09-18      1     1   
251904     105696             0    09  17002.65 2017-09-16      1     1   
251954     74765              0    04  16825.20 2017-04-29      1     1   

                      被保人總資產  具滿期金受益人  具生故保險金受益人  
INSURED_RK Policy_RK                     

In [69]:
X = df.iloc[:,3:]
y = df.iloc[:,0]

In [70]:

X = X.drop(columns=['INJURED_RK', 'Claim_RK','illness_code','claim_settle_dt','POLICY_HOLDER_RK','MATURITY_BENEFICIARY_RK',
       'DEATH_BENEFICIARY_RK', '結案後120天', '結案後180天', '結案後360天','CUST_RK','CLIENT_MARITAL','CLIENT_INCOME','TOTAL_AUM','INSURED_DOB','初次理賠時間'])

In [71]:
X.tail()

,,BundleSubtype2,illness_desc,DiagnosisCode_DESC,REIMBURSED_YR_TW,ternure_m,recency_m,SIN,SIN_his,REG,REG_his,...,stick_level2,cust_group2,被保人是否為事故人,結案月份,累積理賠金額,累積理賠次數,被保人年收,被保人總資產,具滿期金受益人,具生故保險金受益人
INSURED_RK,Policy_RK,,,,,,,,,,,,,,,,,,,,,
251796,74433,6.N意外醫療,摩拖車騎士與汽車，小貨車或箱型車碰撞受傷,20.疾病和死亡的外因,4389.00,321,201,0,0,1,1,...,S09,G4,0,09,4389.00,1,0,1,0,0
251813,78696,6.N意外醫療,摩拖車騎士與汽車，小貨車或箱型車碰撞受傷,20.疾病和死亡的外因,31133.55,263,240,0,0,0,0,...,S09,G4,0,03,31133.55,1,0,0,0,0
251876,57631,6.N意外醫療,涉及其他特定運動和競技之活動,20.疾病和死亡的外因,1470.00,285,13,0,0,1,1,...,S03,G2,0,09,1470.00,1,1,1,1,0
251904,105696,5.N疾病醫療,其他及未明示部位之原位癌,02.腫瘤,17002.65,317,282,0,0,1,1,...,S10,G4,0,09,17002.65,1,1,0,0,0
251954,74765,4.C重大疾病,自體的冠狀動脈粥樣硬化心臟病,09.循環系統疾病,16825.20,297,68,0,0,1,1,...,S07,G4,0,04,16825.20,1,1,0,0,0


In [72]:
X.columns

Index(['BundleSubtype2', 'illness_desc', 'DiagnosisCode_DESC',
       'REIMBURSED_YR_TW', 'ternure_m', 'recency_m', 'SIN', 'SIN_his', 'REG',
       'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb', 'AHb_his', 'AHc',
       'AHc_his', 'AHd', 'AHd_his', 'VIP_CLASS', 'VIP', 'WEALTH_LEVEL',
       'DIGI_FLG', 'TOPCARD', 'GENDER', 'stick_level2', 'cust_group2',
       '被保人是否為事故人', '結案月份', '累積理賠金額', '累積理賠次數', '被保人年收', '被保人總資產', '具滿期金受益人',
       '具生故保險金受益人'],
      dtype='object')

In [73]:
X['VIP_CLASS'] = X['VIP_CLASS'].fillna('0')

In [74]:
X.reset_index(drop=True, inplace=True)

## **處理資料不平衡**
[不平衡資料的二元分類 2：利用抽樣改善模型品質](https://taweihuang.hpd.io/2018/12/30/imbalanced-data-sampling-techniques/) <br>
[Oversampling: SMOTE for binary and categorical data in Python](https://stackoverflow.com/questions/47655813/oversampling-smote-for-binary-and-categorical-data-in-python)

In [77]:
cate = [0,1,2,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,32,33,34,35] # 這些是類別資料，使用smotenc前要先標出來

In [78]:
sm = SMOTENC(categorical_features = cate, random_state=0)
X_res, y_res = sm.fit_resample(X, y)

In [79]:
df1 = X_res
df1['y'] = y_res

In [14]:
df1.to_excel('理賠再購屬性合併2-2_balanced_before_encoding.xlsx')

In [81]:
df1.tail()

,BundleSubtype2,illness_desc,DiagnosisCode_DESC,REIMBURSED_YR_TW,ternure_m,recency_m,SIN,SIN_his,REG,REG_his,...,cust_group2,被保人是否為事故人,結案月份,累積理賠金額,累積理賠次數,被保人年收,被保人總資產,具滿期金受益人,具生故保險金受益人,y
419243,5.N疾病醫療,其他滑倒、摔倒或絆倒和跌落,11.消化系統疾病,13242.732027,127,102,0,0,1,1,...,G2,1,07,2.436667e+04,3,1,0,0,1,1
419244,5.N疾病醫療,齒齦炎及牙周疾病,11.消化系統疾病,180697.282174,287,77,1,1,1,1,...,G0,1,08,2.803968e+06,11,1,1,0,0,1
419245,6.N意外醫療,摩拖車騎士與汽車，小貨車或箱型車碰撞受傷,20.疾病和死亡的外因,2060.993653,268,15,0,0,1,1,...,G2,1,06,2.060994e+03,1,1,0,0,0,1
419246,5.N疾病醫療,分娩及生產時其他併發症,10.呼吸系統疾病,50576.044114,167,12,0,0,1,1,...,G1,1,01,1.207757e+05,3,1,1,0,0,1
419247,5.N疾病醫療,運輸意外事故,11.消化系統疾病,36358.993817,257,5,0,0,1,1,...,G2,1,01,3.635899e+04,1,1,0,0,1,1


In [83]:
df1.columns

Index(['BundleSubtype2', 'illness_desc', 'DiagnosisCode_DESC',
       'REIMBURSED_YR_TW', 'ternure_m', 'recency_m', 'SIN', 'SIN_his', 'REG',
       'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb', 'AHb_his', 'AHc',
       'AHc_his', 'AHd', 'AHd_his', 'VIP_CLASS', 'VIP', 'WEALTH_LEVEL',
       'DIGI_FLG', 'TOPCARD', 'GENDER', 'stick_level2', 'cust_group2',
       '被保人是否為事故人', '結案月份', '累積理賠金額', '累積理賠次數', '被保人年收', '被保人總資產', '具滿期金受益人',
       '具生故保險金受益人', 'y'],
      dtype='object')

## **encoding**

In [93]:
## 要先分訓練跟測試，才能target encoding
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.33, random_state=42)
to_encoding = ['BundleSubtype2', 'illness_desc', 'DiagnosisCode_DESC','VIP_CLASS','WEALTH_LEVEL','stick_level2', 'cust_group2','結案月份']
enc = TargetEncoder(cols=to_encoding)
training_numeric_dataset = enc.fit_transform(X_train, y_train)
testing_numeric_dataset = enc.transform(X_test)

In [97]:
training_numeric_dataset.columns

Index(['BundleSubtype2', 'illness_desc', 'DiagnosisCode_DESC',
       'REIMBURSED_YR_TW', 'ternure_m', 'recency_m', 'SIN', 'SIN_his', 'REG',
       'REG_his', 'ILP', 'ILP_his', 'AHa', 'AHa_his', 'AHb', 'AHb_his', 'AHc',
       'AHc_his', 'AHd', 'AHd_his', 'VIP_CLASS', 'VIP', 'WEALTH_LEVEL',
       'DIGI_FLG', 'TOPCARD', 'GENDER', 'stick_level2', 'cust_group2',
       '被保人是否為事故人', '結案月份', '累積理賠金額', '累積理賠次數', '被保人年收', '被保人總資產', '具滿期金受益人',
       '具生故保險金受益人', 'y'],
      dtype='object')

## **min_max**

In [98]:
scaler = MinMaxScaler()
to_minmax = ['REIMBURSED_YR_TW','ternure_m','recency_m','累積理賠金額','累積理賠次數']
scaler.fit(training_numeric_dataset[to_minmax])
training_numeric_dataset[to_minmax] = scaler.transform(training_numeric_dataset[to_minmax])
testing_numeric_dataset[to_minmax] = scaler.transform(testing_numeric_dataset[to_minmax])

In [99]:
training_numeric_dataset

,BundleSubtype2,illness_desc,DiagnosisCode_DESC,REIMBURSED_YR_TW,ternure_m,recency_m,SIN,SIN_his,REG,REG_his,...,cust_group2,被保人是否為事故人,結案月份,累積理賠金額,累積理賠次數,被保人年收,被保人總資產,具滿期金受益人,具生故保險金受益人,y
161300,0.485099,0.351160,0.342496,0.001623,0.488281,0.154667,0,0,0,1,...,0.221083,1,0.494023,0.004710,0.034483,0,0,0,0,0
194169,0.534082,0.437239,0.316375,0.000581,0.492188,0.032000,0,0,0,1,...,0.796961,1,0.502414,0.000561,0.000000,1,1,0,0,0
174035,0.534082,0.524687,0.619472,0.000239,0.289062,0.208000,0,0,0,0,...,0.221083,1,0.520191,0.000472,0.017241,0,0,1,1,0
275486,0.534082,0.437239,0.316375,0.000422,0.154297,0.013333,0,0,1,1,...,0.788630,1,0.485080,0.000607,0.017241,0,0,0,0,1
389517,0.534082,0.596533,0.619472,0.000055,0.070312,0.013333,0,0,0,1,...,0.788630,1,0.556121,0.000111,0.017241,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,0.534082,0.596533,0.619472,0.010203,0.394531,0.154667,0,0,1,1,...,0.788630,1,0.520191,0.010510,0.017241,1,0,0,0,1
365838,0.485099,0.488131,0.452967,0.010191,0.349609,0.224000,0,0,1,1,...,0.221083,1,0.485080,0.009850,0.000000,1,1,0,0,1
131932,0.534082,0.596533,0.619472,0.001669,0.445312,0.157333,0,0,0,0,...,0.210584,1,0.494023,0.005641,0.051724,1,0,1,1,0
146867,0.485099,0.339660,0.452967,0.000045,0.417969,0.570667,0,0,0,0,...,0.221083,1,0.494023,0.000043,0.000000,0,0,0,0,0


## **儲存最後training, testing data**

In [100]:
training_numeric_dataset.to_excel('training_data2-2.xlsx')
testing_numeric_dataset.to_excel('testing_data2-2.xlsx')